In [2]:
import pandas as pd

In [9]:
# Load dataframes
current = pd.read_csv('data/Current_Sheet.csv')
emissions_limited = pd.read_csv('data/Emissions_Limited_2040.csv')
business_as_usual = pd.read_csv('data/Business_as_Usual_2080.csv')
bau_plus_1degree = pd.read_csv('data/BAU_plus_1degree_2080.csv')

In [13]:
def compare_taxon_lists(df1, df2, Taxon="Taxon"):
    """
    Compare the taxon lists of two datasets.

    Parameters:
    df1 (pd.DataFrame): First dataframe to compare.
    df2 (pd.DataFrame): Second dataframe to compare.currrent
    
    Returns:
    dict: A dictionary with keys 'unique_to_df1', 'unique_to_df2', and 'common' 
          containing taxon lists unique to each dataframe and common to both.
    """
    #remove duplicates
    taxon_df1 = set(df1[Taxon].dropna().unique())
    taxon_df2 = set(df2[Taxon].dropna().unique())

    #compare dataframes
    unique_to_df1 = list(taxon_df1 - taxon_df2)
    unique_to_df2 = list(taxon_df2 - taxon_df1)
    common = list(taxon_df1 & taxon_df2)
    
    return {
        'unique_to_df1': unique_to_df1,
        'unique_to_df2': unique_to_df2,
        'common': common
    }


In [14]:
#input to dataframes names to compare 'Taxon' items
result = compare_taxon_lists(current, business_as_usual)

print(f"""Test results:
Unique to df1 ({len(result['unique_to_df1'])}) 
Unique to df2 ({len(result['unique_to_df2'])})
Common ({len(result['common'])})
""")


Test results:
Unique to df1 (0) 
Unique to df2 (0)
Common (3826)



In [19]:
#input name of dataframe to count
data = current

#Count the occurrences
taxon_counts = data['Taxon'].value_counts()

#Find duplicates > 1)
duplicates = taxon_counts[taxon_counts > 1]

#Create a DataFrame of duplicates
duplicates_df = duplicates.reset_index()
duplicates_df.columns = ['Taxon', 'Count']

#Display
duplicates_df


,Taxon,Count
0,Cotoneaster horizontalis,3
1,Rosa macrophylla,2
2,Rhododendron indicum,2
3,Epimedium leptorrhizum,2
4,Rosa longicuspis,2
5,Astilbe rubra,2
6,Epimedium dolichostemon,2
7,Epimedium davidii,2
8,Cota tinctoria,2
9,Epimedium acuminatum,2
